In [1]:
import xarray as xr
import dask
import dask.array as da
import numpy as np
import pandas as pd
import dask.multiprocessing

import blosc
from timeit import default_timer as timer
from time import sleep

import zarr
import h5py
import gcsfs
from dask_kubernetes import KubeCluster
from dask.distributed import Client, progress, LocalCluster

/home/jovyan/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
start = timer()
end = start
while True:
    print(timer() - start)
    sleep(300)

5.060499825049192e-05
300.09343270499085
600.1924176360044
900.2680356810015
1200.3675593590015
1500.4677192419913
1800.5678693419904
2100.6565664910013
2400.7234654600034
3000.9104211109952
3301.010559589995
3601.104564919995
3901.201312306992
4201.295350796005
4501.395648948994
4801.493601640992
5101.593565617004
5401.682194933004
5701.781561919997
6001.881555947999
6301.981573056997
6602.081768386997
6902.181572082001
7202.281578170994
7502.381779078001
7802.466597252991
8102.566552620992


In [2]:
cluster = KubeCluster(n_workers=120)
cluster

In [4]:
cluster.close()

In [17]:
client = Client(cluster)
client

Client Scheduler: tcp://10.21.55.30:45433 Dashboard: /user/kaipak/proxy/8787/status,Cluster Workers: 120 Cores: 240 Memory: 720.00 GB


In [8]:
fs = gcsfs.GCSFileSystem(project='pangeo-181919', token='cache')
token = fs.session.credentials
fs2 = gcsfs.GCSFileSystem(project='pangeo-181919', token=token)
gcsmap_zarr = gcsfs.GCSMap('pangeo-data/storage-benchmarks/llc4320_zarr', gcs=fs2)

In [9]:
%time llc_zarr = xr.open_zarr(gcsmap_zarr, decode_cf=True, decode_times=False)

CPU times: user 3.21 s, sys: 235 ms, total: 3.45 s
Wall time: 11.7 s


In [22]:
sea_temps = llc_zarr.Theta
print(sea_temps)
print(sea_temps.nbytes / 1024**3)

za = zarr.open_group(gcsmap_zarr)
za['Theta'].info

<xarray.DataArray 'Theta' (time: 417, Z: 90, YC: 2160, XC: 2160)>
dask.array<shape=(417, 90, 2160, 2160), dtype=float32, chunksize=(1, 1, 2160, 2160)>
Coordinates:
  * XC       (XC) float32 -15.489583 -15.46875 -15.447917 -15.427083 ...
  * YC       (YC) float32 -57.001026 -56.989952 -56.978878 -56.9678 ...
  * Z        (Z) float32 -0.5 -1.57 -2.79 -4.185 -5.78 -7.595 -9.66 -12.01 ...
  * time     (time) float64 2.502e+06 2.506e+06 2.509e+06 2.513e+06 ...
Attributes:
    coordinates:    iter
    long_name:      Potential Temperature
    standard_name:  THETA
    units:          degC
652.2982120513916


Name,/Theta
Type,zarr.core.Array
Data type,float32
Shape,"(417, 90, 2160, 2160)"
Chunk shape,"(1, 1, 2160, 2160)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,gcsfs.mapping.GCSMap
No. bytes,700399872000 (652.3G)
Chunks initialized,37530/37530


In [23]:
mean_temps = sea_temps.mean()

In [24]:
%time mean_temps.compute(retries=3)

distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
Future exception was never retrieved
future: <Future finished exception=WebSocketClosedError()>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/tornado/websocket.py", line 808, in wrapper
    yield fut
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1099, in run
    value = future.result()
tornado.iostream.StreamClosedError: Stream is closed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/tornado/gen.py", line 1107, in ru

CPU times: user 1min 26s, sys: 3.81 s, total: 1min 29s
Wall time: 1min 31s


<xarray.DataArray 'Theta' ()>
array(4.127072, dtype=float32)

In [21]:
del mean_temps
del sea_temps

In [ ]:
def randn(shape, frac_nan=None, chunks=None, seed=0):
    rng = np.random.RandomState(seed)
    if chunks is None:
        x = rng.standard_normal(shape)
    else:
        import dask.array as da
        rng = da.random.RandomState(seed)
        x = rng.standard_normal(shape, chunks=chunks)

    if frac_nan is not None:
        inds = rng.choice(range(x.size), int(x.size * frac_nan))
        x.flat[inds] = np.nan

    return x

def randint(low, high=None, size=None, frac_minus=None, seed=0):
    rng = np.random.RandomState(seed)
    x = rng.randint(low, high, size)
    if frac_minus is not None:
        inds = rng.choice(range(x.size), int(x.size * frac_minus))
        x.flat[inds] = -1

    return x

def rand_numpy(nz=None, empty=True):
    """
    Generate random 3D Numpy dataset.

    :params;

    """
    if nz == None:
        nz = getTestConfigValue("num_slices")
    if not nz or nz <= 0:
        raise NotImplementedError("num_slices invalid")
    ny = 1000
    nx = 1000
    dtype = 'f8'
    # Create a dataset
    #dset = f.create_dataset(_DATASET_NAME, shape=(nz,ny,nx), dtype=dtype)

    data = np.random.rand(nz,nx,ny)
    return data

def rand_xarray(nt=None):
    """
    Generate synthetic geoscience-like Xarray dataset filled with random 
    data.
    :param ds: dataset that gets generated.
    :param nt: number of timesteps for data. Primary control over how large
               the dataset is.
    :returns: A synthetic xarray dataset that mimics geodata.
    """

    ds = xr.Dataset()
    if nt == None:
        nt = getTestConfigValue("ntime_slices")
    ny = 1000
    nx = 1000
    block_chunks = {'time': nt / 4,
                             'lon': nx / 3,
                             'lat': ny / 3}

    time_chunks = {'time': int(nt / 36)}

    times = pd.date_range('1970-01-01', periods=nt, freq='D')
    lons = xr.DataArray(np.linspace(0, 360, nx), dims=('lon', ),
                        attrs={'units': 'degrees east',
                               'long_name': 'longitude'})
    lats = xr.DataArray(np.linspace(-90, 90, ny), dims=('lat', ),
                        attrs={'units':'degrees north',
                               'long_name': 'latitude'})
    ds['foo'] = xr.DataArray(randn((nt, nx, ny), frac_nan=0.2),
                             coords={'lon': lons, 'lat': lats,'time': times},
                             dims=('time', 'lon', 'lat'),
                             name='foo', encoding=None,
                             attrs={'units': 'foo units',
                                    'description': 'a description'})
    ds['bar'] = xr.DataArray(randn((nt, nx, ny), frac_nan=0.2),
                             coords={'lon': lons, 'lat': lats, 'time': times},
                             dims=('time', 'lon', 'lat'),
                             name='bar', encoding=None,
                             attrs={'units': 'bar units',
                                    'description': 'a description'})
    ds['baz'] = xr.DataArray(randn((nx, ny), frac_nan=0.2).astype(np.float32),
                             coords={'lon': lons, 'lat': lats},
                             dims=('lon', 'lat'),
                             name='baz', encoding=None,
                             attrs={'units': 'baz units',
                                    'description': 'a description'})

    ds.attrs = {'history': 'created for xarray benchmarking'}

    oinds = {'time': randint(0, nt, 120),
             'lon': randint(0, nx, 20),
             'lat': randint(0, ny, 10)}
    vinds = {'time': xr.DataArray(randint(0, nt, 120), dims='x'),
             'lon': xr.DataArray(randint(0, nx, 120), dims='x'),
             'lat': slice(3, 20)}

    return ds



In [ ]:
np_ds = rand_numpy(64)
np_ds.nbytes/ 1024**2

In [ ]:
xr_ds = rand_xarray(32)
print(xr_ds.nbytes/ 1024**2)

In [ ]:
fs = gcsfs.GCSFileSystem(project='pangeo-181919', token='cache')
token = fs.session.credentials
fs2 = gcsfs.GCSFileSystem(project='pangeo-181919', token=token)
gcsmap_zarr_hoff = gcsfs.GCSMap('pangeo-data/storage-benchmarks/test_zarr_hoff', gcs=fs2)

In [ ]:
chunks = (1, 4000, 4000)
size = (2000, 4000, 4000)
dask_arr = da.random.normal(10, 0.1, size=size, chunks=chunks)
dask_arr.nbytes/ 1024**3

In [ ]:
zarr_ds = zarr.create(dask_arr.shape, chunks=chunks, 
                       dtype=dask_arr.dtype, store=gcsmap_zarr_hoff, overwrite=True)

In [ ]:
%time dask_arr.store(zarr_ds, lock=False)

In [ ]:
for i in range(3):
    dask_arr = da.random.normal(10, 0.1, size=size, chunks=chunks)
    %time dask_arr.store(zarr_ds, lock=False)
    del dask_arr

### NetCDF Testing

In [ ]:
chunksize = 10
%time ds_nc = xr.open_mfdataset('/gcs/storage-benchmarks/llc4320_netcdf/*.nc', decode_cf=False, autoclose=True, chunks={'k': chunksize, 'k_l': chunksize})

In [ ]:
ds_nc

In [ ]:
mean_theta = ds_nc.Theta.mean()
ds_nc.Theta.nbytes / 1024**3

In [ ]:
%time mean_theta.load()

In [ ]:
del mean_theta
del ds_nc

In [ ]:
54.7 / 80

In [ ]:
ds_nc

In [ ]:
ds_nc

In [ ]:
import kube
cluster = kube.Cluster()